In [ ]:
"""
Conduct Data Analysis using OpenAI + Streamlit
"""

In [9]:
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_core.messages import SystemMessage
from dotenv import load_dotenv

load_dotenv()

True

In [10]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            template_format="jinja2",
            template="""Your are a data analyst. Your only role is to provide coding solutions tailored to various data analysis queries using streamlit.io. The GPT should focus on delivering code snippets that directly address the user's analytical needs without any explanation.

The programming language should be Python and framework should be Streamlit. The output should not be in markdown format and only contain Python code.

Dataset Information:{{dataset_info}}"""
        ),
        HumanMessagePromptTemplate.from_template("{query}"),
    ]
)

In [11]:
import pandas as pd

In [12]:
llm = ChatOpenAI(model="gpt-4", temperature=0)

In [13]:
df = pd.read_csv("../california_housing_train.csv")

In [14]:
"""
- Conditional - Does it need code + text, only code, only text ?
- Pass Dataset Info
- Ask for a query
- Get code, install all packages that are needed, or use the tools provided to install packages
- Print the code using Streamlit.
"""

'\n- Conditional - Does it need code + text, only code, only text ?\n- Pass Dataset Info\n- Ask for a query\n- Get code, install all packages that are needed, or use the tools provided to install packages\n- Print the code using Streamlit.\n'